In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, Derive, log, bioDraws, MonteCarlo
import math
import random
import biogeme.results as res
from random import randint
import os, inspect
from sklearn.utils import shuffle
import numpy as np


import warnings
warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync/CFS')

In [2]:
region_df = pd.read_csv('CFS2017_austin_forML.csv')
factor_df = pd.read_csv('data_unitcost_by_zone_cfs2017.csv')
sctg_group = pd.read_csv('SCTG_Groups_revised.csv')
print(region_df.columns)
region_df.head(5)

Index(['SHIPMT_ID', 'ORIG_STATE', 'ORIG_MA', 'ORIG_CFS_AREA', 'DEST_STATE',
       'DEST_MA', 'DEST_CFS_AREA', 'NAICS', 'QUARTER', 'SCTG', 'MODE',
       'SHIPMT_VALUE', 'SHIPMT_WGHT', 'SHIPMT_DIST_GC', 'SHIPMT_DIST_ROUTED',
       'TEMP_CNTL_YN', 'EXPORT_YN', 'EXPORT_CNTRY', 'HAZMAT', 'WGT_FACTOR',
       'mode_agg5', 'bulk', 'fuel_fert', 'interm_food', 'mfr_goods', 'other',
       'commodity', 'naics2', 'naics_name', 'wholesale', 'mfring', 'mining',
       'retail', 'info', 'management', 'transwarehouse', 'SHIPMT_WGHT_TON',
       'value_density', 'SHIPMT_DIST', 'choice', 'travel_time', 'ship_cost'],
      dtype='object')


,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,...,retail,info,management,transwarehouse,SHIPMT_WGHT_TON,value_density,SHIPMT_DIST,choice,travel_time,ship_cost
0,6,48,12420,48-12420,36,160,36-160,4541,2,40,...,1,0,0,0,0.0020,151.250000,1348,3,24.000000,128.766412
1,9,17,176,17-176,48,41700,48-41700,333,1,40,...,0,0,0,0,0.0190,50.473684,1057,3,120.000000,25.355610
2,13,48,206,48-206,48,99999,48-99999,322,2,28,...,0,0,0,0,0.3420,0.982456,517,2,23.953846,79.686000
3,44,13,122,13-122,48,99999,48-99999,4242,2,21,...,0,0,0,0,0.0005,3.000000,729,3,120.000000,13.052765
4,66,48,99999,48-99999,48,99999,48-99999,4239,2,26,...,0,0,0,0,69.2885,0.106670,419,5,55.045455,1132.243379


In [3]:
region_df.loc[:, ['SHIPMT_VALUE', 'SHIPMT_WGHT', 'value_density']].head(10)

,SHIPMT_VALUE,SHIPMT_WGHT,value_density
0,605,4,151.250000
1,1918,38,50.473684
2,672,684,0.982456
3,3,1,3.000000
4,14782,138577,0.106670
5,3428,40420,0.084809
6,2984,11,271.272727
7,35808,13747,2.604786
8,814,810,1.004938
9,123,6,20.500000


In [3]:
# clean the data

df_clean = region_df.loc[region_df.EXPORT_YN == 'N'] # Cleans out international exports

def removeOutliers(sName, df):
    # Computing IQR
    Q1 = df[sName].quantile(0.25)
    Q3 = df[sName].quantile(0.75)
    IQR = Q3 - Q1

    # Filtering Values between Q1-1.5IQR and Q3+1.5IQR
    filtered = df.query(sName + '<= (@Q3 + 3 * @IQR)')
    
    return filtered

df_clean = removeOutliers('SHIPMT_WGHT', df_clean)
df_clean = removeOutliers('SHIPMT_DIST', df_clean)
print(len(df_clean))

206269


In [4]:
sctg_group.head(5)
df_clean = pd.merge(df_clean, sctg_group,
                    left_on = 'SCTG',
                    right_on = 'SCTG_Code',
                    how = 'left')

In [5]:
var_to_keep = ['SHIPMT_ID', 'ORIG_CFS_AREA', 'DEST_CFS_AREA', 'NAICS', 'naics_name',
               'SCTG', 'SCTG_Group', 'SHIPMT_VALUE', 'SHIPMT_WGHT', 'SHIPMT_DIST_GC', 'value_density']

df_clean_choice_model = df_clean[var_to_keep]
dist_matrix = df_clean_choice_model.groupby(['ORIG_CFS_AREA', 'DEST_CFS_AREA'])[['SHIPMT_DIST_GC']].mean()
dist_matrix = dist_matrix.reset_index()
print(dist_matrix.head(5))


  ORIG_CFS_AREA DEST_CFS_AREA  SHIPMT_DIST_GC
0        01-142      48-12420      690.556962
1        01-142      48-41700      755.881579
2        01-142      48-99999      698.684659
3        01-380      48-12420      586.800000
4        01-380      48-41700      631.050000


In [6]:
unique_naics = df_clean_choice_model.NAICS.unique()
df_clean_choice_model.loc[:, 'chosen'] = 0
supplier_selection_set = None
for naics in unique_naics:
    print(naics)
    all_suppliers = \
    df_clean_choice_model.loc[df_clean_choice_model['NAICS'] == naics]   
    chunk_of_suppliers = np.array_split(all_suppliers, 10)
    #print(chunk_of_suppliers[0].head(5))
    for i in range(10):
        chunk = chunk_of_suppliers[i]
        chunk.loc[:, 'chosen'] = 1
        #print(chunk.columns)
        shipment_to_match = chunk[['SHIPMT_ID', 'DEST_CFS_AREA', 
                                   'NAICS', 'naics_name', 'SCTG', 'SCTG_Group', 'SHIPMT_WGHT']]
        selected_shipment = chunk.SHIPMT_ID.unique()
        non_chosen_set = \
        all_suppliers.loc[~all_suppliers['SHIPMT_ID'].isin(selected_shipment)]
        non_chosen_set = non_chosen_set[['ORIG_CFS_AREA', 'NAICS', 
                                         'naics_name', 'SCTG', 'value_density', 'chosen']]
        non_chosen_set = pd.merge(shipment_to_match, non_chosen_set,
                                  on = ['NAICS', 'naics_name', 'SCTG'], how = 'left')
        non_chosen_set = pd.merge(non_chosen_set, dist_matrix, 
                                  on = ['ORIG_CFS_AREA', 'DEST_CFS_AREA'], how = 'left')
        non_chosen_set.loc[:, 'SHIPMT_DIST_GC'].fillna(3000, inplace = True)
        non_chosen_set.loc[:, 'value_density'] = \
        non_chosen_set.loc[:,'value_density'].fillna(non_chosen_set.groupby('SCTG')['value_density'].transform('mean'))
        non_chosen_set.loc[:, 'SHIPMT_VALUE'] = \
        non_chosen_set.loc[:, 'SHIPMT_WGHT'] * non_chosen_set.loc[:, 'value_density']
        non_chosen_set = non_chosen_set.groupby('SHIPMT_ID').sample(n=9, replace = True, random_state=1)
        #print(non_chosen_set.head(5))
        combined_set = pd.concat([chunk, non_chosen_set])
        combined_set = combined_set.sort_values('SHIPMT_ID')
#         print(combined_set.head(20))
        supplier_selection_set = pd.concat([supplier_selection_set, combined_set])
#         break
        
#     break

4541
333
322
4242
334
314
4244
337
4234
4235
4931
311
4241
4239
4233
321
4248
332
4236
312
323
339
4232
4238
326
4249
313
4243
4237
336
335
325
4231
4247
551114
327
5111
315
4246
316
45431
331
212
324
4245


In [7]:
# creat label
supplier_selection_set.loc[:, 'SHIPMT_VALUE'] = \
supplier_selection_set.loc[:, 'SHIPMT_VALUE'].fillna(supplier_selection_set.groupby('SHIPMT_ID')['SHIPMT_VALUE'].transform('mean'))
supplier_selection_set = shuffle(supplier_selection_set)
print(len(supplier_selection_set))
supplier_selection_set = supplier_selection_set.sort_values('SHIPMT_ID')
supplier_selection_set['alternative'] = \
supplier_selection_set.groupby('SHIPMT_ID').cumcount() + 1
supplier_selection_set['choice'] = 0
supplier_selection_set.loc[supplier_selection_set['chosen'] == 1, 'choice'] = \
supplier_selection_set.loc[supplier_selection_set['chosen'] == 1, 'alternative']

supplier_selection_set.head(10)

2062690


,SHIPMT_ID,ORIG_CFS_AREA,DEST_CFS_AREA,NAICS,naics_name,SCTG,SCTG_Group,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,value_density,chosen,alternative,choice
1932,6,36-408,36-160,4541,Retail,40,4,112.000000,4,3000.000000,28.000000,0.0,1,0
847,6,26-266,36-160,4541,Retail,40,4,226.000000,4,3000.000000,56.500000,0.0,2,0
1061,6,48-12420,36-160,4541,Retail,40,4,7128.000000,4,1360.047619,1782.000000,0.0,3,0
715,6,22-99999,36-160,4541,Retail,40,4,50.666667,4,3000.000000,12.666667,0.0,4,0
1669,6,48-12420,36-160,4541,Retail,40,4,9.200000,4,1360.047619,2.300000,0.0,5,0
1096,6,47-314,36-160,4541,Retail,40,4,104.000000,4,3000.000000,26.000000,0.0,6,0
905,6,06-348,36-160,4541,Retail,40,4,81.846154,4,3000.000000,20.461538,0.0,7,0
235,6,48-206,36-160,4541,Retail,40,4,7.200000,4,3000.000000,1.800000,0.0,8,0
0,6,48-12420,36-160,4541,Retail,40,4,605.000000,4,1348.000000,151.250000,1.0,9,9
1791,6,48-12420,36-160,4541,Retail,40,4,972.000000,4,1360.047619,243.000000,0.0,10,0


In [8]:
# append capacity
factor_df.head(5)
supplier_selection_set = pd.merge(supplier_selection_set, factor_df,
                                 left_on = ['ORIG_CFS_AREA', 'SCTG'],
                                 right_on = ['ORIG_CFS_AREA', 'Commodity_SCTG'],
                                 how = 'left')
supplier_selection_set.loc[:, 'Capacity'] = \
supplier_selection_set.loc[:, 'Capacity'].fillna(supplier_selection_set.groupby('SHIPMT_ID')['Capacity'].transform('mean'))
supplier_selection_set.head(5)

,SHIPMT_ID,ORIG_CFS_AREA,DEST_CFS_AREA,NAICS,naics_name,SCTG,SCTG_Group,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,value_density,chosen,alternative,choice,Commodity_SCTG,Capacity,UnitCost
0,6,36-408,36-160,4541,Retail,40,4,112.000000,4,3000.000000,28.000000,0.0,1,0,40.0,1.198220e+06,25958.470176
1,6,26-266,36-160,4541,Retail,40,4,226.000000,4,3000.000000,56.500000,0.0,2,0,40.0,2.463173e+05,11776.628317
2,6,48-12420,36-160,4541,Retail,40,4,7128.000000,4,1360.047619,1782.000000,0.0,3,0,40.0,6.693005e+05,5997.918722
3,6,22-99999,36-160,4541,Retail,40,4,50.666667,4,3000.000000,12.666667,0.0,4,0,40.0,1.874652e+05,10714.272533
4,6,48-12420,36-160,4541,Retail,40,4,9.200000,4,1360.047619,2.300000,0.0,5,0,40.0,6.693005e+05,5997.918722


In [9]:
# convert long data to wide
choice = supplier_selection_set.loc[supplier_selection_set['chosen'] == 1, 
                                     ['SHIPMT_ID',	'NAICS', 'SCTG_Group', 'choice']]

factor_1 = pd.pivot_table(supplier_selection_set, values='SHIPMT_VALUE', index=['SHIPMT_ID'],
                    columns=['alternative'], aggfunc=np.mean)
factor_1.columns = ['value_' + str(i+1) for i in range(10)]
factor_1 = factor_1.reset_index()
# factor_1.head(5)

factor_2 = pd.pivot_table(supplier_selection_set, values='SHIPMT_DIST_GC', index=['SHIPMT_ID'],
                    columns=['alternative'], aggfunc=np.mean)
factor_2.columns = ['distance_' + str(i+1) for i in range(10)]
# factor_2 = factor_2.fillna(1)
factor_2 = factor_2.reset_index()

factor_3 = pd.pivot_table(supplier_selection_set, values='Capacity', index=['SHIPMT_ID'],
                    columns=['alternative'], aggfunc=np.mean)
factor_3.columns = ['capacity_' + str(i+1) for i in range(10)]
# factor_2 = factor_2.fillna(1)
factor_3 = factor_3.reset_index()
# factor_2.head(5)

destination_choice_data_wide = pd.merge(choice, factor_1, 
                                        on = 'SHIPMT_ID', how = 'left')
destination_choice_data_wide = pd.merge(destination_choice_data_wide, factor_2, 
                                        on = 'SHIPMT_ID', how = 'left')
destination_choice_data_wide = pd.merge(destination_choice_data_wide, factor_3, 
                                        on = 'SHIPMT_ID', how = 'left')
destination_choice_data_wide.head(5)

,SHIPMT_ID,NAICS,SCTG_Group,choice,value_1,value_2,value_3,value_4,value_5,value_6,...,capacity_1,capacity_2,capacity_3,capacity_4,capacity_5,capacity_6,capacity_7,capacity_8,capacity_9,capacity_10
0,6,4541,4,9,112.000000,226.000000,7128.000000,50.666667,9.200000,104.000000,...,1.198220e+06,246317.29510,6.693005e+05,187465.21770,6.693005e+05,4.217153e+05,7.535399e+06,2.930314e+06,6.693005e+05,669300.47235
1,9,333,4,3,26.482789,680.865979,1918.000000,1026.000000,147.976471,15580.000000,...,4.255420e+05,187465.21770,2.384661e+06,315165.62540,3.923336e+05,1.198220e+06,2.588460e+05,8.432752e+05,3.128944e+05,843275.16865
2,13,322,4,5,350.626415,1309.549091,612.564935,515.826692,672.000000,279.020979,...,2.957800e+06,303062.27595,8.637301e+05,863730.06385,1.967717e+06,3.030623e+05,1.967717e+06,1.967717e+06,2.957800e+06,863730.06385
3,44,4242,4,9,52.500000,6.000000,45.000000,23.750000,15.714286,19.785714,...,1.754967e+05,175496.74325,1.754967e+05,175496.74325,2.713819e+04,1.754967e+05,1.770116e+04,1.754967e+05,1.910093e+05,158554.75675
4,130,334,4,1,2984.000000,55.000000,1724.800000,729.666667,88.000000,253.000000,...,6.693005e+05,669300.47235,9.699524e+04,843275.16865,3.425309e+05,6.693005e+05,7.535399e+06,8.432752e+05,6.693005e+05,96995.24305


In [10]:
# add availability
destination_choice_data_wide.loc[:, 'av_1'] = 1
destination_choice_data_wide.loc[:, 'av_2'] = 1
destination_choice_data_wide.loc[:, 'av_3'] = 1
destination_choice_data_wide.loc[:, 'av_4'] = 1
destination_choice_data_wide.loc[:, 'av_5'] = 1
destination_choice_data_wide.loc[:, 'av_6'] = 1
destination_choice_data_wide.loc[:, 'av_7'] = 1
destination_choice_data_wide.loc[:, 'av_8'] = 1
destination_choice_data_wide.loc[:, 'av_9'] = 1
destination_choice_data_wide.loc[:, 'av_10'] = 1

In [15]:
sctg_groups = destination_choice_data_wide.SCTG_Group.unique()
for sctg in sctg_groups:
    print(sctg)
    destination_choice_data_selected = \
    destination_choice_data_wide.loc[destination_choice_data_wide['SCTG_Group'] == sctg]
    database = db.Database('destination_choice', destination_choice_data_selected)
    globals().update(database.variables)
    database.fullData
    # define parameters
    B_VALUE = Beta('B_VALUE', 0, None, None, 0)
    B_DISTANCE = Beta('B_DISTANCE', 0, None, None, 0)
#     B_CAPACITY = Beta('B_CAPACITY', 0, None, None, 0)

    # B_VALUE_S = Beta('B_VALUE_S', 0.0001, None, None, 0)
    # B_DISTANCE_S = Beta('B_DISTANCE_S', 0.01, None, None, 0)
    # Define a random parameter with a normal distribution, designed to be used
    # for quasi Monte-Carlo simulation with Halton draws (base 5).
    # B_VALUE_RND = B_VALUE + B_VALUE_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON5')
    # B_DISTANCE_RND = B_DISTANCE + B_DISTANCE_S * bioDraws('B_DISTANCE_RND', 'NORMAL_HALTON5')


    V1 = B_VALUE * value_1 + B_DISTANCE * distance_1
    V2 = B_VALUE * value_2 + B_DISTANCE * distance_2
    V3 = B_VALUE * value_3 + B_DISTANCE * distance_3
    V4 = B_VALUE * value_4 + B_DISTANCE * distance_4
    V5 = B_VALUE * value_5 + B_DISTANCE * distance_5
    V6 = B_VALUE * value_6 + B_DISTANCE * distance_6
    V7 = B_VALUE * value_7 + B_DISTANCE * distance_7
    V8 = B_VALUE * value_8 + B_DISTANCE * distance_8 
    V9 = B_VALUE * value_9 + B_DISTANCE * distance_9 
    V10 = B_VALUE * value_10 + B_DISTANCE * distance_10

    V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5,
        6: V6, 7: V7, 8: V8, 9: V9, 10: V10}

    av = {1: av_1, 2: av_2, 3: av_3, 4: av_4, 5: av_5, 
          6: av_6, 7: av_7, 8: av_8, 9: av_9, 10: av_10}
    # logprob = models.loglogit(V, None, choice)
    logprob = models.loglogit(V, av, choice)
    biogeme = bio.BIOGEME(database, logprob)
    biogeme.modelName = 'supplier_selection_austin'
    biogeme.calculateNullLoglikelihood(av)
    # Estimate the parameters
    results = biogeme.estimate()

    # Get the results in a pandas table
    pandasResults = results.getEstimatedParameters()
    goodness_of_fit = results.getGeneralStatistics()
    print('estimation results')
    print(pandasResults)
    print(goodness_of_fit['Rho-square for the null model'][0])

4
estimation results
                   Value       Std err     t-test  p-value  Rob. Std err  \
B_DISTANCE -4.233376e-04  4.923373e-06 -85.985276      0.0  4.659824e-06   
B_VALUE    -2.611525e-07  1.496006e-08 -17.456652      0.0  8.129014e-09   

            Rob. t-test  Rob. p-value  
B_DISTANCE   -90.848417           0.0  
B_VALUE      -32.125976           0.0  
0.016699913285588175
5
estimation results
               Value       Std err     t-test  p-value  Rob. Std err  \
B_DISTANCE -0.000638  2.208184e-05 -28.901335      0.0  1.925020e-05   
B_VALUE    -0.000006  4.093236e-07 -14.904613      0.0  3.511148e-07   

            Rob. t-test  Rob. p-value  
B_DISTANCE   -33.152632           0.0  
B_VALUE      -17.375542           0.0  
0.02941690596878266
3
estimation results
               Value       Std err     t-test  p-value  Rob. Std err  \
B_DISTANCE -0.000662  2.265379e-05 -29.236368      0.0  2.432686e-05   
B_VALUE    -0.000015  6.970616e-07 -21.683533      0.0  8.151309e-

In [13]:
sctg_groups = destination_choice_data_wide.SCTG_Group.unique()
for sctg in sctg_groups:
    print(sctg)
    destination_choice_data_selected = \
    destination_choice_data_wide.loc[destination_choice_data_wide['SCTG_Group'] == sctg]
    database = db.Database('destination_choice', destination_choice_data_selected)
    globals().update(database.variables)
    database.fullData
    # define parameters
    B_VALUE = Beta('B_VALUE', 0, None, None, 0)
    B_DISTANCE_LOW = Beta('B_DISTANCE_LOW', 0, None, None, 0)
    B_DISTANCE_HIGH = Beta('B_DISTANCE_HIGH', 0, None, None, 0)
    B_DISTANCE_CONST = Beta('B_DISTANCE_CONST', 0, None, None, 0)
#     B_CAPACITY = Beta('B_CAPACITY', 0, None, None, 0)

    # B_VALUE_S = Beta('B_VALUE_S', 0.0001, None, None, 0)
    # B_DISTANCE_S = Beta('B_DISTANCE_S', 0.01, None, None, 0)
    # Define a random parameter with a normal distribution, designed to be used
    # for quasi Monte-Carlo simulation with Halton draws (base 5).
    # B_VALUE_RND = B_VALUE + B_VALUE_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON5')
    # B_DISTANCE_RND = B_DISTANCE + B_DISTANCE_S * bioDraws('B_DISTANCE_RND', 'NORMAL_HALTON5')


    V1 = B_VALUE * value_1 + B_DISTANCE_LOW * distance_1 * (distance_1 <= 500) + \
    B_DISTANCE_HIGH * distance_1 * (distance_1 > 500) + B_DISTANCE_CONST * (distance_1 > 500)
    V2 = B_VALUE * value_2 + B_DISTANCE_LOW * distance_2 * (distance_2 <= 500) + \
    B_DISTANCE_HIGH * distance_2 * (distance_2 > 500) + B_DISTANCE_CONST * (distance_2 > 500)
    V3 = B_VALUE * value_3 + B_DISTANCE_LOW * distance_3 * (distance_3 <= 500) + \
    B_DISTANCE_HIGH * distance_3 * (distance_3 > 500) + B_DISTANCE_CONST * (distance_3 > 500)
    V4 = B_VALUE * value_4 + B_DISTANCE_LOW * distance_4 * (distance_4 <= 500) + \
    B_DISTANCE_HIGH * distance_4 * (distance_4 > 500) + B_DISTANCE_CONST * (distance_4 > 500)
    V5 = B_VALUE * value_5 + B_DISTANCE_LOW * distance_5 * (distance_5 <= 500) + \
    B_DISTANCE_HIGH * distance_5 * (distance_5 > 500) + B_DISTANCE_CONST * (distance_5 > 500)
    V6 = B_VALUE * value_6 + B_DISTANCE_LOW * distance_6 * (distance_6 <= 500) + \
    B_DISTANCE_HIGH * distance_6 * (distance_6 > 500) + B_DISTANCE_CONST * (distance_6 > 500)
    V7 = B_VALUE * value_7 + B_DISTANCE_LOW * distance_7 * (distance_7 <= 500) + \
    B_DISTANCE_HIGH * distance_7 * (distance_7 > 500) + B_DISTANCE_CONST * (distance_7 > 500)
    V8 = B_VALUE * value_8 + B_DISTANCE_LOW * distance_8 * (distance_8 <= 500) + \
    B_DISTANCE_HIGH * distance_8 * (distance_8 > 500) + B_DISTANCE_CONST * (distance_8 > 500) 
    V9 = B_VALUE * value_9 + B_DISTANCE_LOW * distance_9 * (distance_9 <= 500) + \
    B_DISTANCE_HIGH * distance_9 * (distance_9 > 500) + B_DISTANCE_CONST * (distance_9 > 500)
    V10 = B_VALUE * value_10 + B_DISTANCE_LOW * distance_10 * (distance_10 <= 500) + \
    B_DISTANCE_HIGH * distance_10 * (distance_10 > 500) + B_DISTANCE_CONST * (distance_10 > 500)

    V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5,
        6: V6, 7: V7, 8: V8, 9: V9, 10: V10}

    av = {1: av_1, 2: av_2, 3: av_3, 4: av_4, 5: av_5, 
          6: av_6, 7: av_7, 8: av_8, 9: av_9, 10: av_10}
    # logprob = models.loglogit(V, None, choice)
    logprob = models.loglogit(V, av, choice)
    biogeme = bio.BIOGEME(database, logprob)
    biogeme.modelName = 'supplier_selection_austin'
    biogeme.calculateNullLoglikelihood(av)
    # Estimate the parameters
    results = biogeme.estimate()

    # Get the results in a pandas table
    pandasResults = results.getEstimatedParameters()
    goodness_of_fit = results.getGeneralStatistics()
    print('estimation results')
    print(pandasResults)
    print(goodness_of_fit['Rho-square for the null model'][0])

4
estimation results
                     Value       Std err      t-test  p-value  Rob. Std err  \
B_DISTANCE_CONST  1.716342  1.369240e-02  125.349983      0.0  1.262773e-02   
B_DISTANCE_HIGH  -0.001285  9.310162e-06 -137.996492      0.0  7.065632e-06   
B_DISTANCE_LOW   -0.001056  4.749986e-05  -22.232680      0.0  7.084405e-05   
B_VALUE          -0.000008  1.370703e-07  -58.065250      0.0  4.821786e-07   

                  Rob. t-test  Rob. p-value  
B_DISTANCE_CONST   135.918535           0.0  
B_DISTANCE_HIGH   -181.833662           0.0  
B_DISTANCE_LOW     -14.906673           0.0  
B_VALUE            -16.506381           0.0  
0.06589261828314597
5
estimation results
                     Value       Std err     t-test  p-value  Rob. Std err  \
B_DISTANCE_CONST  1.372076  5.902333e-02  23.246336      0.0  5.324806e-02   
B_DISTANCE_HIGH  -0.001401  4.263384e-05 -32.869296      0.0  2.920881e-05   
B_DISTANCE_LOW   -0.001797  1.356003e-04 -13.254380      0.0  1.946658e-04   


In [13]:
# destination_choice_data_selected = destination_choice_data_wide.loc[destination_choice_data_wide['NAICS'] == 333]
destination_choice_data_selected = destination_choice_data_wide.head(50000)
database = db.Database('destination_choice', destination_choice_data_selected)
globals().update(database.variables)
database.fullData
# define parameters
B_VALUE = Beta('B_VALUE', 0, None, None, 0)
B_DISTANCE = Beta('B_DISTANCE', 0, None, None, 0)

V1 = B_VALUE * value_1 + B_DISTANCE * distance_1
V2 = B_VALUE * value_2 + B_DISTANCE * distance_2
V3 = B_VALUE * value_3 + B_DISTANCE * distance_3
V4 = B_VALUE * value_4 + B_DISTANCE * distance_4
V5 = B_VALUE * value_5 + B_DISTANCE * distance_5
V6 = B_VALUE * value_6 + B_DISTANCE * distance_6
V7 = B_VALUE * value_7 + B_DISTANCE * distance_7
V8 = B_VALUE * value_8 + B_DISTANCE * distance_8
V9 = B_VALUE * value_9 + B_DISTANCE * distance_9
V10 = B_VALUE * value_10 + B_DISTANCE * distance_10

V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5,
    6: V6, 7: V7, 8: V8, 9: V9, 10: V10}

av = {1: av_1, 2: av_2, 3: av_3, 4: av_4, 5: av_5, 
      6: av_6, 7: av_7, 8: av_8, 9: av_9, 10: av_10}

logprob = models.loglogit(V, av, choice)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'supplier_selection'
biogeme.calculateNullLoglikelihood(av)
# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
goodness_of_fit = results.getGeneralStatistics()
print('estimation results')
print(pandasResults)
print(goodness_of_fit['Rho-square-bar for the init. model'][0])

estimation results
                   Value       Std err     t-test       p-value  Rob. Std err  \
B_DISTANCE -9.903068e-05  7.164454e-06 -13.822501  0.000000e+00  7.471980e-06   
B_VALUE    -1.066834e-07  1.597369e-08  -6.678694  2.410805e-11  5.632429e-09   

            Rob. t-test  Rob. p-value  
B_DISTANCE   -13.253606           0.0  
B_VALUE      -18.940914           0.0  
0.0023153786631519013


In [12]:
validationData = database.split(slices=5)

validation_results = biogeme.validate(results, validationData)

for slide in validation_results:
    print(
        f'Log likelihood for {slide.shape[0]} validation data: '
        f'{slide["Loglikelihood"].sum()}'
    )

Log likelihood for 10000 validation data: -22521.86062041235
Log likelihood for 10000 validation data: -22447.934767976683
Log likelihood for 10000 validation data: -22186.99405266812
Log likelihood for 10000 validation data: -22063.792608900138
Log likelihood for 10000 validation data: -21961.317473647527


In [20]:
print(logprob)

_bioLogLogitFullChoiceSet(1:((B_VALUE(-1.0950943913870096e-05) * value_1) + (B_DISTANCE(-0.00044825343739563134) * distance_1)), 2:((B_VALUE(-1.0950943913870096e-05) * value_2) + (B_DISTANCE(-0.00044825343739563134) * distance_2)), 3:((B_VALUE(-1.0950943913870096e-05) * value_3) + (B_DISTANCE(-0.00044825343739563134) * distance_3)), 4:((B_VALUE(-1.0950943913870096e-05) * value_4) + (B_DISTANCE(-0.00044825343739563134) * distance_4)), 5:((B_VALUE(-1.0950943913870096e-05) * value_5) + (B_DISTANCE(-0.00044825343739563134) * distance_5)), 6:((B_VALUE(-1.0950943913870096e-05) * value_6) + (B_DISTANCE(-0.00044825343739563134) * distance_6)), 7:((B_VALUE(-1.0950943913870096e-05) * value_7) + (B_DISTANCE(-0.00044825343739563134) * distance_7)), 8:((B_VALUE(-1.0950943913870096e-05) * value_8) + (B_DISTANCE(-0.00044825343739563134) * distance_8)), 9:((B_VALUE(-1.0950943913870096e-05) * value_9) + (B_DISTANCE(-0.00044825343739563134) * distance_9)), 10:((B_VALUE(-1.0950943913870096e-05) * value_1

In [11]:
destination_choice_data_wide.loc[destination_choice_data_wide.isna().any(axis=1)]

,SHIPMT_ID,NAICS,choice,value_1,value_2,value_3,value_4,value_5,value_6,value_7,...,av_1,av_2,av_3,av_4,av_5,av_6,av_7,av_8,av_9,av_10
